# 🏔️ MODIS Albedo Analysis - Athabasca Glacier
## Setup and Configuration

**Université du Québec à Trois-Rivières - Projet de Maîtrise**

This notebook sets up the environment and configuration for analyzing MODIS albedo data from the Athabasca Glacier in Alberta, Canada.

---

## 📦 Import Required Libraries

In [ ]:
import ee
import geemap
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import json
import os
from datetime import datetime, date
import ipywidgets as widgets
from IPython.display import display, HTML

# Configure matplotlib for better plots
plt.rcParams['figure.figsize'] = (12, 8)
plt.rcParams['font.size'] = 10
sns.set_style("whitegrid")

print("✅ Libraries imported successfully!")

## 🔐 Google Earth Engine Authentication

**First time users:** Run `ee.Authenticate()` in a separate cell if you haven't authenticated before.

In [ ]:
# Uncomment and run this line if you need to authenticate for the first time
# ee.Authenticate()

# Initialize Earth Engine
ee.Initialize()
print("✅ Google Earth Engine initialized successfully!")

## 🏔️ Glacier Configuration

Load the Athabasca Glacier boundary from GeoJSON file and set up the region of interest.

In [ ]:
# Load the glacier mask GeoJSON
geojson_path = '../Athabasca_mask_2023 (1).geojson'

with open(geojson_path, 'r') as f:
    athabasca_geojson = json.load(f)

# Convert GeoJSON to Earth Engine geometry
geometries = []
for feature in athabasca_geojson['features']:
    if feature['geometry']['type'] == 'MultiPolygon':
        for polygon in feature['geometry']['coordinates']:
            geometries.append(ee.Geometry.Polygon(polygon))
    elif feature['geometry']['type'] == 'Polygon':
        geometries.append(ee.Geometry.Polygon(feature['geometry']['coordinates']))

# Create unified glacier region of interest
athabasca_roi = ee.FeatureCollection(geometries).geometry()

# Weather station coordinates
ATHABASCA_STATION = [-117.245, 52.214]
station_point = ee.Geometry.Point(ATHABASCA_STATION)

# Calculate glacier area
try:
    glacier_area = athabasca_roi.area().divide(1e6).getInfo()  # Convert to km²
    glacier_centroid = athabasca_roi.centroid().getInfo()
    
    print(f"🏔️ GLACIER INFORMATION")
    print(f"📏 Surface area: {glacier_area:.2f} km²")
    print(f"📍 Center coordinates: {glacier_centroid['coordinates'][1]:.3f}°N, {glacier_centroid['coordinates'][0]:.3f}°W")
    print(f"🌡️ Weather station: {ATHABASCA_STATION[1]:.3f}°N, {ATHABASCA_STATION[0]:.3f}°W")
except Exception as e:
    print(f"⚠️ Could not calculate glacier properties: {e}")

print("✅ Glacier geometry loaded successfully!")

## 📡 MODIS Data Configuration

Define MODIS collections and analysis parameters.

In [ ]:
# MODIS Collections (Version 6.1)
MODIS_COLLECTIONS = {
    'snow_terra': 'MODIS/061/MOD10A1',    # Terra Snow Cover
    'snow_aqua': 'MODIS/061/MYD10A1',     # Aqua Snow Cover
    'broadband': 'MODIS/061/MCD43A3'      # Combined BRDF/Albedo
}

# Analysis periods
PERIODS = {
    'recent': ('2020-01-01', '2024-10-31'),
    'fire_years': ('2017-01-01', '2021-12-31'),
    'decade': ('2015-01-01', '2024-10-31'),
    'full_recent': ('2019-01-01', '2024-10-31')
}

# Fire years for annotations
FIRE_YEARS = {
    2017: "Feux BC",
    2018: "Feux BC", 
    2019: "Feux AB",
    2020: "Feux CA/US",
    2023: "Feux record"
}

# Default analysis parameters
DEFAULT_SCALE = 500  # MODIS native resolution
DEFAULT_QA_THRESHOLD = 1  # Quality threshold (≤1 = good quality)

print("📡 MODIS CONFIGURATION")
print(f"   • Terra Snow: {MODIS_COLLECTIONS['snow_terra']}")
print(f"   • Aqua Snow: {MODIS_COLLECTIONS['snow_aqua']}")
print(f"   • Broadband: {MODIS_COLLECTIONS['broadband']}")
print(f"   • Native resolution: {DEFAULT_SCALE}m")
print(f"   • Quality threshold: QA ≤ {DEFAULT_QA_THRESHOLD}")
print("✅ MODIS configuration ready!")

## 🛠️ Helper Functions

Define utility functions for the analysis.

In [ ]:
def create_date_widget(description="Select Date:", default_date=None):
    """Create a date picker widget"""
    if default_date is None:
        default_date = date(2023, 8, 15)
    
    return widgets.DatePicker(
        description=description,
        value=default_date,
        disabled=False,
        style={'description_width': 'initial'}
    )

def create_period_widget():
    """Create a period selection widget"""
    return widgets.Dropdown(
        options=list(PERIODS.keys()),
        value='full_recent',
        description='Period:',
        disabled=False,
        style={'description_width': 'initial'}
    )

def display_period_info(period_key):
    """Display information about a selected period"""
    start, end = PERIODS[period_key]
    print(f"📅 Period: {period_key.title()}")
    print(f"   Start: {start}")
    print(f"   End: {end}")
    
    # Calculate duration
    start_dt = datetime.strptime(start, '%Y-%m-%d')
    end_dt = datetime.strptime(end, '%Y-%m-%d')
    duration = (end_dt - start_dt).days / 365.25
    print(f"   Duration: {duration:.1f} years")

print("✅ Helper functions defined!")

## 🔬 Test Connection and Data Availability

Verify that we can access MODIS data for the glacier region.

In [ ]:
# Test data access
test_start = '2023-08-01'
test_end = '2023-08-31'

print(f"🔍 Testing data access for {test_start} to {test_end}...")

try:
    # Test Terra snow collection
    terra_collection = ee.ImageCollection(MODIS_COLLECTIONS['snow_terra']) \
        .filterBounds(athabasca_roi) \
        .filterDate(test_start, test_end)
    
    terra_count = terra_collection.size().getInfo()
    
    # Test Aqua snow collection
    aqua_collection = ee.ImageCollection(MODIS_COLLECTIONS['snow_aqua']) \
        .filterBounds(athabasca_roi) \
        .filterDate(test_start, test_end)
    
    aqua_count = aqua_collection.size().getInfo()
    
    # Test broadband collection
    broad_collection = ee.ImageCollection(MODIS_COLLECTIONS['broadband']) \
        .filterBounds(athabasca_roi) \
        .filterDate(test_start, test_end)
    
    broad_count = broad_collection.size().getInfo()
    
    print(f"📊 DATA AVAILABILITY TEST RESULTS:")
    print(f"   • Terra (MOD10A1): {terra_count} images")
    print(f"   • Aqua (MYD10A1): {aqua_count} images")
    print(f"   • Broadband (MCD43A3): {broad_count} images")
    print(f"   • Total snow images: {terra_count + aqua_count}")
    
    if terra_count > 0 or aqua_count > 0:
        print("✅ Data access successful!")
    else:
        print("⚠️ No data found for test period")
        
except Exception as e:
    print(f"❌ Error accessing data: {e}")
    print("Please check your Earth Engine authentication.")

## 📋 Next Steps

Configuration is complete! You can now proceed to:

1. **02_Interactive_Mapping.ipynb** - Interactive maps with date selectors
2. **03_Data_Processing.ipynb** - MODIS data extraction and processing
3. **04_Time_Series_Analysis.ipynb** - Albedo time series analysis
4. **05_Melt_Season_Analysis.ipynb** - Specialized melt season analysis
5. **06_Visualization.ipynb** - Advanced plotting and visualization

---

**🎯 Variables now available in this session:**
- `athabasca_roi` - Glacier region of interest
- `MODIS_COLLECTIONS` - MODIS data collections
- `PERIODS` - Analysis time periods
- `FIRE_YEARS` - Fire year annotations
- Helper functions for widgets and utilities